.gnt to .png代码搬运自 http://www.codeceo.com/article/tensorflow-and-handwritten-chinese-character.html
拷贝注意数据文件目录

## .gnt to .png 

In [ ]:
import os
import numpy as np
import struct
from PIL import Image

data_dir = './dataset'
train_data_dir = os.path.join(data_dir, 'HWDB1.1trn_gnt')
test_data_dir = os.path.join(data_dir, 'HWDB1.1tst_gnt')

def read_from_gnt_dir(gnt_dir=train_data_dir):
    def one_file(f):
        header_size = 10
        while True:
            header = np.fromfile(f, dtype='uint8', count=header_size)
            if not header.size: break
            sample_size = header[0] + (header[1]<<8) + (header[2]<<16) + (header[3]<<24)
            tagcode = header[5] + (header[4]<<8)
            width = header[6] + (header[7]<<8)
            height = header[8] + (header[9]<<8)
            if header_size + width*height != sample_size:
                break
            image = np.fromfile(f, dtype='uint8', count=width*height).reshape((height, width))
            yield image, tagcode
    for file_name in os.listdir(gnt_dir):
        if file_name.endswith('.gnt'):
            file_path = os.path.join(gnt_dir, file_name)
            with open(file_path, 'rb') as f:
                for image, tagcode in one_file(f):
                    yield image, tagcode
char_set = set()
for _, tagcode in read_from_gnt_dir(gnt_dir=train_data_dir):
    tagcode_unicode = struct.pack('>H', tagcode).decode('gb2312')
    char_set.add(tagcode_unicode)
char_list = list(char_set)
char_dict = dict(zip(sorted(char_list), range(len(char_list))))
print len(char_dict)
import pickle
f = open('char_dict', 'wb')
pickle.dump(char_dict, f)
f.close()
train_counter = 0
test_counter = 0

for image, tagcode in read_from_gnt_dir(gnt_dir=train_data_dir):
    tagcode_unicode = struct.pack('>H', tagcode).decode('gb2312')
    im = Image.fromarray(image)
    dir_name = './dataset/' + '%0.5d'%char_dict[tagcode_unicode]
    if not os.path.exists(dir_name):
        os.mkdir(dir_name)
    im.convert('RGB').save(dir_name+'/' + str(train_counter) + '.png')
    train_counter += 1

转换过后，每个汉字图片放在不同的目录中，
可通过`char_dict`字典查看目录索引

In [ ]:

digit_label = [char_dict['一'],
              char_dict['二'],
              char_dict['三'],
              char_dict['四'],
              char_dict['五'],
              char_dict['六'],
              char_dict['七'],
              char_dict['八'],
              char_dict['九'],
              char_dict['十']]
digit_label

In [ ]:
# 将目录索引转化为长度为5的数字，便于直接图区
digit_dir = []
for digit in digit_label:
    digit_str = list(str(digit))
    while len(digit_str) < 5:
        digit_str.insert(0,'0')
    digit_str = "".join(digit_str)
    digit_dir.append(digit_str)
digit_dir

In [ ]:
# 将图片与标签，配对生成字典
digit_dict = dict(zip(digit_dir,[1,2,3,4,5,6,7,8,9,10]))

In [ ]:
# 生成所有的图片的文件路径和对应索引
image_files=[]
label_arr=[]
dataPath = "./dataset"
for char in digit_dir:
    for image in os.listdir(os.path.join(dataPath,char)):
        image_files.append(os.path.join(dataPath,char,image))
        label_arr.append(int(digit_dict[char]))

In [ ]:
# 存取变量便于模型读取图片
joblib.dump(image_files,'image_files.pkl')
joblib.dump(label_arr,'label_arr.pkl')